In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = [
    "DataFrames",
    "StatsBase",
    "StatsPlots",
    "uCSV",
    "ProgressMeter",
    "Distances",
    "Clustering",
    "Colors",
    "MultivariateStats",
    "Dates",
    "CategoricalArrays",
    "GLM",
    "Statistics",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
base_dir = dirname(pwd())
data_directory = joinpath(base_dir, "data")
results_dir = mkpath(joinpath(base_dir, "results"))

In [ ]:
sample_directories = readdir(joinpath(data_directory, "samples"), join=true)

In [ ]:
kraken_dbs = filter(x -> isdir(x), readdir("$(homedir())/workspace/kraken", join=true))

In [ ]:
# kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_20231009"
kraken_db = "k2_pluspfp_20231009"

In [ ]:
kraken_reports = String[]
for sample_directory in sample_directories
    trim_galore_directories = filter(x -> occursin(r"trimgalore$", x), readdir(sample_directory, join=true))
    for trim_galore_directory in trim_galore_directories
        # @show trim_galore_directory
        kraken_directories = filter(x -> occursin(r"kraken", x), readdir(trim_galore_directory, join=true))
        for kraken_directory in kraken_directories
            reports = filter(x -> occursin(r"kraken-report\.tsv$", x), readdir(kraken_directory, join=true))
            append!(kraken_reports, reports)
        end
    end
end
kraken_reports = filter(x -> occursin(kraken_db, x), kraken_reports)

In [ ]:
taxon_levels = Mycelia.list_ranks()

In [ ]:
# i = 1
# i = 2
# i = 3
# i = 4
# i = 5
# i = 6
# i = 7
i = 8
# i = 9

(taxon_index, taxon_level) = collect(enumerate(taxon_levels))[i]
println("$(taxon_index) - $(taxon_level)")
rank_table = Mycelia.list_rank(taxon_level)

In [ ]:
# turn me into a function that accepts a rank table and a list of kraken reports as well as an output
rank_taxids = Set(rank_table[!, "taxid"])
cross_sample_taxon_report_table = DataFrames.DataFrame()
ProgressMeter.@showprogress for kraken_report in kraken_reports
    report_table = Mycelia.read_kraken_report(kraken_report)
    taxon_level_report = report_table[map(x -> x in rank_taxids, report_table[!, "ncbi_taxonid"]), :]
    taxon_level_report[!, "sample_identifier"] .= basename(kraken_report)
    append!(cross_sample_taxon_report_table, taxon_level_report)
end
cross_sample_taxon_report_table

In [ ]:
cross_sample_taxon_report_table[cross_sample_taxon_report_table[!, "ncbi_taxonid"] .== 11102, :]

In [ ]:
any(cross_sample_taxon_report_table[cross_sample_taxon_report_table[!, "ncbi_taxonid"] .== 11102, "number_of_fragments_at_or_below_taxon"] .> 0)